# Running a Q# program on Azure Quantum from Python

This notebook demonstrates how to run a Q# program on Azure Quantum from Python.

## Import Q# operations

Q# operations can be imported from source files and used from Python.

This folder contains a file called [SimpleGrover.qs](../edit/SimpleGrover.qs), which contains a number of Q# operations.

These operations can be listed using the `qsharp.get_workspace_operations()` command:

In [ ]:
import qsharp
import qsharp.azure

qsharp.get_workspace_operations()

Q# operations can also be implemented inline.

For example, the following cell compiles a Q# operation called `SearchForMarkedInput` that implements [Grover's search algorithm](https://docs.microsoft.com/quantum/tutorials/search):

In [ ]:
search_operation = qsharp.compile('''
    open Microsoft.Quantum.Arrays;
    open Microsoft.Quantum.Diagnostics;
    open Microsoft.Quantum.Measurement;
    open Microsoft.Quantum.Samples.SimpleGrover;

    operation SearchForMarkedInput(nQubits : Int, idxMarked : Int) : Result[] {
        using (qubits = Qubit[nQubits]) {
            // Initialize a uniform superposition over all possible inputs.
            PrepareUniform(qubits);
            // The search itself consists of repeatedly reflecting about the
            // marked state and our start state, which we can write out in Q#
            // as a for loop.
            for (idxIteration in 0..NIterations(nQubits) - 1) {
                ReflectAboutMarked(idxMarked, qubits);
                ReflectAboutUniform(qubits);
            }
            // Output diagnostic information.
            DumpMachine();
            // Measure and return the answer.
            return ForEach(MResetZ, qubits);
        }
    }
''')

## Local simulation of Q# operations

Using `simulate()`, you can invoke the built-in functionality to simulate a quantum operation locally and return the result. You can specify any operation that has been defined in the notebook or that has been imported from a separate file.

For example, to simulate the `SearchForMarkedInput` operation with `nQubits=2` and `idxMarked=3`, run:

`search_operation.simulate(nQubits=2, idxMarked=3)`

To use `nQubits=3` and `idxMarked=5` instead, simply change the corresponding parameters:

`search_operation.simulate(nQubits=3, idxMarked=5)`

## Running Q# programs via Azure Quantum

First, find the resource ID of your Azure Quantum workspace. You can copy/paste this from the top-right corner of your Quantum Workspace page in Azure Portal.

Next, you can run `qsharp.azure.connect()` to connect to the workspace and display the list of provisioned targets that support running Q# programs. If you are prompted to login, be sure to use the same account you used to create your Azure Quantum workspace. For example:

`qsharp.azure.connect(resourceId='<resource ID of workspace>')`

Now, use `qsharp.azure.target()` to specify the target you'd like to use for job submission. To use the `ionq.simulator` target, run:

`qsharp.azure.target('ionq.simulator')`

To submit a job, use `qsharp.azure.submit()` along with the Q# operation and any parameters required by that operation. The `qsharp.azure.submit()` command will return immediately after the job is created.

For example, to submit the `SearchForMarkedInput` operation with `nQubits=2` and `idxMarked=3`, run:

`qsharp.azure.submit(search_operation, nQubits=2, idxMarked=3, jobName="SearchForMarkedInput (2 qubits)")`

To use `nQubits=3` and `idxMarked=5` instead, simply change the corresponding parameters:

`qsharp.azure.submit(search_operation, nQubits=3, idxMarked=5, jobName="SearchForMarkedInput (3 qubits)")`

## Viewing job status and results

Running `qsharp.azure.status()` will return the status of the most recently submitted job in this session.

Once the job has completed, use `qsharp.azure.output()` to return the result.

You can also get the status of all jobs by using `%azure.jobs`. Providing a parameter to this command will filter to just the jobs containing that string. For example, you can query for the status of all jobs containing the string `SearchForMarkedInput` by running:

`qsharp.azure.jobs('SearchForMarkedInput')`

In [ ]:
for job in qsharp.azure.jobs('SearchForMarkedInput'):
    print(f'{job.id}\t{job.target}\t{job.name}\t  Status = {job.status}')

We can see that some jobs were previously submitted to the `ionq.simulator` and `ionq.qpu` targets. By specifying the job ID, we can view the status and results of any of those jobs as well. For example, to view the results of job ID `3e7454f2-17b5-4318-a9e8-d0152193d6ef`, run:

`qsharp.azure.output('3e7454f2-17b5-4318-a9e8-d0152193d6ef')`